# Creating Detached Prompt
This notebook will cover creating a detached prompt in Cloud and CP4D environments.

In [10]:
#install teh required libraries
!pip install --upgrade ibm-aigov-facts-client | tail -n 1
!pip install --upgrade ibm-watson-openscale | tail -n 1

In [11]:
use_cpd = False #set this to true if using cp4d environment

### The below values are needed if using Cloud Environment

In [18]:
CLOUD_API_KEY="YOUR_IBM_CLOUD_API_KEY"
PROJECT_ID ="PROJECT_ID_OF_THE_PROJECT_WHERE_YOU_WANT_TO_STORE_THE_DETACHED_PROMPT"

### The below values are needed if using CP4D Environment

In [13]:
CPD_URL = "YOUR_CP4D_URL"
CPD_USERNAME = "CP4D Username"
CPD_PASSWORD = "CP4D Password"
WOS_SERVICE_INSTANCE_ID = None #change this if you have multiple openscale instances.

### Setup the Factsheet client

In [14]:
from ibm_aigov_facts_client import AIGovFactsClient
from ibm_aigov_facts_client import CloudPakforDataConfig

facts_client = None

if(use_cpd): 
    creds=CloudPakforDataConfig(
    service_url=CPD_URL,
    username=CPD_USERNAME,
    password=CPD_PASSWORD)
    
    facts_client = AIGovFactsClient(
        cloud_pak_for_data_configs=creds,
        container_id=PROJECT_ID,
        container_type="project",
        disable_tracing=True
    )
else:
    facts_client = AIGovFactsClient(
        api_key=CLOUD_API_KEY,
        container_id=PROJECT_ID,
        container_type="project",
        disable_tracing=True
    )

### Prompt Template Details

In [15]:
# define parameters for PromptTemplate
prompt_variables = {"question": "", "context" :""}
prompt_input = "You are an assistant for question-answering tasks. Use three sentences maximum and keep the answer concise.\nQuestion: {question}\nInformation: {context}"
input_prefix= ""
output_prefix= "Answer:"

### Model Details

In [16]:
#support task_ids are
#1) classification
#2) extraction
#3) generation
#4) question_answering
#5) retrieval_augmented_generation
#6) summarization
task_id = "retrieval_augmented_generation"
name = "Sample Model"
description = "This is a sample model to show a detached prompt"
model_id = "sample/granite"

In [17]:
from ibm_aigov_facts_client import DetachedPromptTemplate, PromptTemplate

prompt_template = PromptTemplate(
    input=prompt_input,
    prompt_variables=prompt_variables,
    input_prefix=input_prefix,
    output_prefix=output_prefix
)

detached_information = DetachedPromptTemplate(
    prompt_id="detached_prompt",
    model_id=model_id,
    model_provider="IBM Granite",
    model_name="Sample Granite",
    model_url="https://sample.ibm.com",
    prompt_url="https://sample.ibm.com/prompt?123",
    prompt_additional_info={"model_owner": "IBM"}
 
)

pta_details = facts_client.assets.create_detached_prompt(
    model_id=model_id,
    task_id=task_id,
    name=name,
    description=description,
    prompt_details=prompt_template,
    detached_information=detached_information
)
project_pta_id = pta_details.to_dict()["asset_id"]

2025/03/13 07:32:11 INFO : ------------------------------ Detached Prompt Creation Started ------------------------------
2025/03/13 07:32:12 INFO : The detached prompt with ID cb141986-9a59-443d-a392-089811429805 was created successfully in container_id 7fc59cf6-a651-4980-b35f-7334c04e1eff.
